# Serving ONNX model with MXNet Model Server

This tutorial dmeonstrates how to serve an ONNX model with MXNet Model Server. We'll use a pre-trained SqueezeNet model from ONNX model zoo. The same example can be easily applied to other ONNX models.

Frameworks used in this tutorial:
* [MXNet Model Server](https://github.com/awslabs/mxnet-model-server) that uses [MXNet](http://mxnet.io)
* [ONNX](https://onnx.ai)

## Installing pre-requisites:
Next we'll install the pre-requisites:
* [ONNX](https://github.com/onnx/onnx)
* [MXNetModelServer](https://github.com/awslabs/mxnet-model-server)

In [ ]:
!conda install -y -c conda-forge onnx
!pip install mxnet-model-server

## Downloading a pre-trained ONNX model

Let's go ahead and download a aqueezenet onnx model into a new directory.

In [ ]:
!mkdir squeezenet
%cd squeezenet
!curl -O https://s3.amazonaws.com/model-server/models/onnx-squeezenet/squeezenet.onnx

## Inspecting the ONNX model

Let's make sure the exported ONNX model is well formed

In [ ]:
import onnx

# Load the ONNX model
model = onnx.load("squeezenet.onnx")

# Check that the IR is well formed, identified issues will error out
onnx.checker.check_model(model)

## Packaging the ONNX model for serving with MXNet Model Server (MMS)

To serve the ONNX model with MMS, we will first need to prepare some files that will be bundled into a **Model Archive**.  
The Model Archive containes everything MMS needs to setup endpoints and serve the model. 

The files needed are:
* squeezenet.onnx - the ONNX model file
* signature.json - defining the input and output of the model
* synset.txt - defining the set of classes the model was trained on, and returned by the model

In [ ]:
# Let's go ahead and download the files we need:
!curl -O https://s3.amazonaws.com/model-server/models/onnx-squeezenet/signature.json
!curl -O https://s3.amazonaws.com/model-server/models/onnx-squeezenet/synset.txt

In [ ]:
# Let's take a peek into the **signature.json** file:
!cat signature.json

In [ ]:
# Let's take a peek into the synset.txt file:
!head -n 10 synset.txt

In [ ]:
# Let's package everything up into a Model Archive bundle
!mxnet-model-export --model-name squeezenet --model-path .
!ls -l squeezenet.model

## Serving the Model Archive with MXNet Model Server
Now that we have the **Model Archive**, we can start the server and ask it to setup HTTP endpoints to serve the model, emit real-time operational metrics and more.

We'll also test the server's endpoints.

In [ ]:
# Spawning a new process to run the server
import subprocess as sp
server = sp.Popen("mxnet-model-server --models squeezenet=squeezenet.model", shell=True)

In [ ]:
# Check out the health endpoint
!curl http://127.0.0.1:8080/ping

In [ ]:
# Download an image Trying out image classification
!curl -O https://s3.amazonaws.com/model-server/inputs/kitten.jpg
!curl -X POST http://127.0.0.1:8080/squeezenet/predict -F "input_0=@kitten.jpg"

In [ ]:
# Lastly, we'll terminate the server
server.terminate()